In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
# Gross
import sys
sys.path.append("..")

In [110]:
import random
from world_rando import generate
from rom_tools.rom_manager import RomManager, region_map_locs
from rom_tools import rom_data_structures, leveldata_utils
from world_rando.room_dtypes import convert_rooms
from rom_tools.address import Address
from encoding import free_space
from world_rando import map_viz
from typing import cast

In [111]:
from rom_tools import rom_edit

In [112]:
rom_m = RomManager("../../roms/Super Metroid (JU) [!] (patched - Subversion 1.2).smc", "../../roms/sm_test_edit.smc")
# rom_m = RomManager("../../roms/sm_clean.smc", "../../roms/sm_test_edit.smc")

looks like subversion


In [117]:
print("Parsing original rooms")
parsed_rooms = rom_m.parse()

Parsing original rooms
Parsing: save_station_0x44c5
Parsing: room_header_0x79330
Parsing: door_list_0x7b420
Parsing: door_0x1c96c
Parsing: room_header_0x79bb8
Parsing: door_list_0x7aa7b
Parsing: door_0x1a0f4
Parsing: room_header_0x797c2
Parsing: door_list_0x7a8c5
Parsing: door_0x1cf00
Parsing: room_header_0x796ff
Parsing: door_list_0x7b1c2
Parsing: door_0x1cee8
Parsing: room_header_0x7979b
Parsing: door_list_0x7af38
Parsing: door_0x1cec4
Parsing: room_header_0x7926d
Parsing: door_list_0x7ab25
Parsing: door_0x19fbc
Parsing: room_header_0x798d3
Parsing: door_list_0x7a919
Parsing: door_0x19f80
Parsing: room_header_0x796d8
Parsing: door_list_0x7af82
Parsing: door_0x19f98
Parsing: room_header_0x7eabb
Parsing: door_list_0x7b1e0
Parsing: door_0x1c558
Parsing: room_header_0x7bb5b
Parsing: door_list_0x7a83e
Parsing: door_0x1c5b8
Parsing: room_header_0x7da9b
Parsing: door_list_0x7ae68
Parsing: door_0x1c600
Parsing: room_header_0x7a37d
Parsing: door_list_0x7b564
Parsing: door_0x1c654
Done Parsing

AssertionError: failed to parse 0x7fee8

In [85]:
rooms: list[rom_data_structures.RoomHeader] = []
for key in parsed_rooms.keys():
    if key.startswith("room_header"):
        # print(key)
        rooms.append(cast(rom_data_structures.RoomHeader, parsed_rooms[key]))

In [86]:
for room_header in rooms:
    state_chooser = cast(rom_data_structures.StateChooser, room_header.state_chooser)
    default = cast(rom_data_structures.RoomState, state_chooser.default)
    level_data = default.level_data
    # print(type(level_data))
    level_array = cast(leveldata_utils.LevelArrays, level_data.level_array)
    # for row in level_array.bts:
    #     print(row)
    print(f"{room_header.name}: {len(level_array.bts)} {len(level_array.bts[0])}")

room_header_0x791f8: 144 80
room_header_0x792fd: 80 80
room_header_0x7990d: 96 48
room_header_0x799bd: 16 112
room_header_0x79969: 64 16
room_header_0x79938: 16 16
room_header_0x79ad9: 64 192
room_header_0x79b9d: 48 16
room_header_0x79c35: 16 16
room_header_0x79fe5: 16 16
room_header_0x7a011: 80 32
room_header_0x7a051: 16 16
room_header_0x7a22a: 16 16
room_header_0x79c5e: 48 32
room_header_0x79c89: 16 16
room_header_0x79bc8: 48 32
room_header_0x79c07: 32 16
room_header_0x79cb3: 112 112
room_header_0x79d19: 80 160
room_header_0x79d9c: 64 16
room_header_0x79dc7: 16 48
room_header_0x79b5b: 32 144
room_header_0x7a0a4: 48 16
room_header_0x79e11: 32 32
room_header_0x79e52: 128 64
room_header_0x79e9f: 128 48
room_header_0x79f11: 16 32
room_header_0x79f64: 48 48
room_header_0x7a1ad: 32 16
room_header_0x7a1d8: 16 16
room_header_0x7a107: 16 16
room_header_0x797b5: 16 16
room_header_0x7975c: 48 32
room_header_0x796ba: 48 144
room_header_0x799f9: 64 128
room_header_0x7dede: 48 96
room_header_0x7de

In [87]:
room_header = rooms[5]
level_array = cast(leveldata_utils.LevelArrays, room_header.state_chooser.default.level_data.level_array)
for row_i in range(16):
    for col in level_array.layer1:
        tile = col[row_i]
        print(tile.tile_type or " ", end="  ")
    print()

8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  
8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  
8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  
8  8  8                                8  8  8  
8  8  8                                8  8  8  
8  8  8                                8  8  8  
8                                         12  9  
8                                         13  9  
8                                         13  9  
8                                         13  9  
8  8  8  8  8  8  8  9  9  8  8  8  8  8  8  8  
8  8  8  8  8  8  8        8  8  8  8  8  8  8  
8  8  8  8  8  8  8        8  8  8  8  8  8  8  
8  8  8  8  8  8  8        8  8  8  8  8  8  8  
8  8  8  8  8  8  8        8  8  8  8  8  8  8  
8  8  8  8  8  8  8  9  9  8  8  8  8  8  8  8  
